In [1]:
import pandas as pd
import os
import sqlite3 as sql
import numpy as np

from functions import fetch_data

In [2]:
import warnings
warnings.simplefilter('ignore')

Data for the backtesting environment will come from Yahoo Finance

In [92]:
print('https://finance.yahoo.com/lookup/')

https://finance.yahoo.com/lookup/


In [3]:
path = 'C:\\Users\\William\\Downloads'

pull files from downloads to save as pandas dataframes

In [83]:
dataframes = {}
files = ['FB.csv', 'AAPL.csv', 'NFLX.csv', 'GOOG.csv', 'ES=F.csv']

for file in files:
    
    df = pd.read_csv(os.path.join(path, file))
    
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')

    df.sort_values('date', inplace=True)
    df.drop('adj_close', axis=1, inplace=True)

    if file == 'ES=F.csv':
        name = 'sp500_df'
    else:
        name = file[:-4].lower() + '_df'
    
    dataframes[name] = df

dataframes.keys()

dict_keys(['fb_df', 'aapl_df', 'nflx_df', 'goog_df', 'sp500_df'])

Drop observations before 2009

In [84]:
dataframes['aapl_df'] = dataframes['aapl_df'].loc[7078:].dropna(axis=0)

In [85]:
dataframes['nflx_df'] = dataframes['nflx_df'].loc[1664:].dropna(axis=0)

In [86]:
dataframes['goog_df'] = dataframes['goog_df'].loc[1100:].dropna(axis=0)

In [87]:
dataframes['sp500_df'] = dataframes['sp500_df'].loc[2577:].dropna(axis=0)

Function for building the db

In [88]:
def build_sql(conn, all_dfs):

    for name, df in all_dfs.items():

        df.to_sql(name, con=conn, index=False, if_exists='replace')
    
    conn.commit()

Function for querying/testing the db

In [89]:
def query(conn, name):
    
    c = conn.cursor()
    
    c.execute(f"""
        SELECT * 
        FROM {name};
        """)
    
    df = pd.DataFrame(c.fetchall(), columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['date'])
    
    print(df.head())

I like to wrap SQL functions in try-except clauses to avoid certain errors with the .db file

In [90]:
connection = sql.connect('ticker_data.db')

try:
    build_sql(connection, dataframes)
    #query(connection, 'fb_df')
except Exception as e:
    print('EXCEPTION', e)

connection.close()

In [91]:
connection = sql.connect('ticker_data.db')

try:
    query(connection, 'sp500_df')
except Exception as e:
    print('EXCEPTION', e)

connection.close()

        date    open    high     low   close     volume
0 2008-12-30  870.25  889.00  868.75  888.25   777046.0
1 2009-01-02  900.00  932.75  893.50  925.50  1095813.0
2 2009-01-05  924.75  934.25  916.50  927.50  1606235.0
3 2009-01-06  927.25  942.75  923.50  930.50  1912293.0
4 2009-01-07  930.25  931.75  898.75  905.25  1961214.0


In [3]:
fetch_data('fb_df').head()

,date,open,high,low,close,volume
0,2012-05-18,42.049999,45.000000,38.000000,38.230000,573576400
1,2012-05-21,36.529999,36.660000,33.000000,34.029999,168192700
2,2012-05-22,32.610001,33.590000,30.940001,31.000000,101786600
3,2012-05-23,31.370001,32.500000,31.360001,32.000000,73600000
4,2012-05-24,32.950001,33.209999,31.770000,33.029999,50237200
